<a href="https://colab.research.google.com/github/zosimanoz/head_pose_metalearning/blob/master/FO_MAML_head_pose_meta_learning_maml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1196614533194079912
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15501623393593013000
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/My Drive/")

In [ ]:
import json
import os
import random
import urllib.request
from collections import defaultdict
from copy import deepcopy
from statistics import mean, stdev
from urllib.error import HTTPError

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from IPython.display import set_matplotlib_formats
from PIL import Image
from torchvision import transforms
from torchvision.datasets import CIFAR100, SVHN
from tqdm.auto import tqdm

In [ ]:
!pip install higher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytorch-lightning
import pytorch_lightning as pl
pl.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 34.2 MB/s 
     |████████████████████████████████| 419 kB 59.1 MB/s 
     |████████████████████████████████| 5.9 MB 54.3 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


'1.7.2'

In [ ]:
# PyTorch Lightning

try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Import tensorboard
%load_ext tensorboard

pl.__version__

'1.7.2'

In [ ]:
CHECKPOINT_PATH = "JOURNAL_ARTICLE_SECOND_ORDER_FINALIZED_10shot"

In [ ]:
class FewShotBatchSampler:
    def __init__(self,dataset_targets, N_way, K_shot, include_query=False, shuffle=True, shuffle_once=False):
        super().__init__()
        self.dataset_targets = torch.tensor(dataset_targets)
        self.N_way = N_way
        self.K_shot = K_shot
        self.shuffle = shuffle
        self.include_query = include_query
        if self.include_query:
            self.K_shot *= 2
        self.batch_size = self.N_way * self.K_shot  # Number of overall images per batch

        # Organize examples by class
        self.classes = torch.unique(self.dataset_targets).tolist()
        self.num_classes = len(self.classes)
        self.indices_per_class = {}
        self.batches_per_class = {}  # Number of K-shot batches that each class can provide
        for c in self.classes:
            self.indices_per_class[c] = torch.where(self.dataset_targets == c)[0]
            self.batches_per_class[c] = self.indices_per_class[c].shape[0] // self.K_shot

        # Create a list of classes from which we select the N classes per batch
        self.iterations = sum(self.batches_per_class.values()) // self.N_way
        self.class_list = [c for c in self.classes for _ in range(self.batches_per_class[c])]
        if shuffle_once or self.shuffle:
            self.shuffle_data()
        else:
            # For testing, we iterate over classes instead of shuffling them
            sort_idxs = [
                i + p * self.num_classes for i, c in enumerate(self.classes) for p in range(self.batches_per_class[c])
            ]
            self.class_list = np.array(self.class_list)[np.argsort(sort_idxs)].tolist()

    def shuffle_data(self):
        # Shuffle the examples per class
        for c in self.classes:
            perm = torch.randperm(self.indices_per_class[c].shape[0])
            self.indices_per_class[c] = self.indices_per_class[c][perm]
        # Shuffle the class list from which we sample. Note that this way of shuffling
        # does not prevent to choose the same class twice in a batch. However, for
        # training and validation, this is not a problem.
        random.shuffle(self.class_list)

    def __iter__(self):
        # Shuffle data
        if self.shuffle:
            self.shuffle_data()

        # Sample few-shot batches
        start_index = defaultdict(int)
        for it in range(self.iterations):
            class_batch = self.class_list[it * self.N_way : (it + 1) * self.N_way]  # Select N classes for the batch
            index_batch = []
            for c in class_batch:  # For each class, select the next K examples and add them to the batch
                index_batch.extend(self.indices_per_class[c][start_index[c] : start_index[c] + self.K_shot])
                start_index[c] += self.K_shot
            if self.include_query:  # If we return support+query set, sort them so that they are easy to split
                index_batch = index_batch[::2] + index_batch[1::2]
            yield index_batch

    def __len__(self):
        return self.iterations

In [ ]:
# class TaskBatchSampler(object):

#     def __init__(self, dataset_targets, batch_size, N_way, K_shot, include_query=False, shuffle=True):
#         super().__init__()
#         self.batch_sampler = FewShotBatchSampler(dataset_targets, N_way, K_shot, include_query, shuffle)
#         self.task_batch_size = batch_size
#         self.local_batch_size = self.batch_sampler.batch_size

#     def __iter__(self):
#         # Aggregate multiple batches before returning the indices
#         batch_list = []
#         for batch_idx, batch in enumerate(self.batch_sampler):
#             batch_list.extend(batch)
#             if (batch_idx+1) % self.task_batch_size == 0:
#                 yield batch_list
#                 batch_list = []

#     def __len__(self):
#         return len(self.batch_sampler)//self.task_batch_size

#     def get_collate_fn(self):
#         # Returns a collate function that converts one big tensor into a list of task-specific tensors
#         def collate_fn(item_list):
#             imgs = torch.stack([torch.from_numpy(img) for img, gt, folder, target in item_list], dim=0)
#             targets = torch.stack([torch.from_numpy(np.fromstring(target, dtype=int, sep=' ')) for img, gt, folder, target in item_list], dim=0)
#             yprAnglesGt = torch.stack([torch.from_numpy(gt) for img, gt, folder, target in item_list], dim=0)

#             imgs = imgs.chunk(self.task_batch_size, dim=0)
#             targets = targets.chunk(self.task_batch_size, dim=0)
#             yprAnglesGt = yprAnglesGt.chunk(self.task_batch_size, dim=0)


#             return imgs, targets, yprAnglesGt
#         return collate_fn

In [ ]:
class TaskBatchSampler(object):

    def __init__(self, dataset_targets, batch_size, N_way, K_shot, include_query=False, shuffle=True):
        super().__init__()
        self.batch_sampler = FewShotBatchSampler(dataset_targets, N_way, K_shot, include_query, shuffle)
        self.task_batch_size = batch_size
        self.local_batch_size = self.batch_sampler.batch_size

    def __iter__(self):
        # Aggregate multiple batches before returning the indices
        batch_list = []
        for batch_idx, batch in enumerate(self.batch_sampler):
            batch_list.extend(batch)
            if (batch_idx+1) % self.task_batch_size == 0:
                yield batch_list
                batch_list = []

    def __len__(self):
        return len(self.batch_sampler)//self.task_batch_size

    def get_collate_fn(self):
        # Returns a collate function that converts one big tensor into a list of task-specific tensors
        def collate_fn(item_list):
            imgs = torch.stack([torch.from_numpy(img) for img, gt, folder, target in item_list], dim=0)
            targets = torch.stack([torch.from_numpy(np.fromstring(target, dtype=int, sep=' ')) for img, gt, folder, target in item_list], dim=0)
            yprAnglesGt = torch.stack([torch.from_numpy(gt) for img, gt, folder, target in item_list], dim=0)

            imgs = imgs.chunk(self.task_batch_size, dim=0)
            targets = targets.chunk(self.task_batch_size, dim=0)
            yprAnglesGt = yprAnglesGt.chunk(self.task_batch_size, dim=0)


            return list(zip(imgs, targets, yprAnglesGt))
        return collate_fn

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
filename = 'trained_models/embeddings_full.npz'
n_embeddings = np.load(filename, allow_pickle=True)
arr = n_embeddings['arr_0']
classes = [str(i).zfill(2) for i in range(1,21)]
n_arr = []
for i in range(arr.shape[0]):
  a = arr[i]
  for j in classes:
    if a[2][13:15] == j:
      # d = scaler.fit_transform(arr[i][1].reshape(-1, 1))
      li = np.array([arr[i][0], arr[i][1], arr[i][2], j])
  n_arr.append(li)
n_arr = np.array(n_arr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
classes = [str(i).zfill(2) for i in range(1,21)]

In [ ]:
# use first 15 person as training and rest 5 person as test

train_set = []
val_set = []
test_set = []

for i in range(n_arr.shape[0]):
  if int(n_arr[i][3]) <= 15:
    train_set.append(n_arr[i])
  else:
    val_set.append(n_arr[i])

train_set_n = np.array(train_set)
val_set_n = np.array(val_set[:2350])
test_set_n = np.array(val_set[2350:])


print(train_set_n.shape)
print(val_set_n.shape)
print(test_set_n.shape)

# get classes of train and test set
train_targets = []
val_targets = []
test_targets = []
for i in range(train_set_n.shape[0]):
    train_targets.append(int(train_set_n[i][3]))
for i in range(val_set_n.shape[0]):
    val_targets.append(int(val_set_n[i][3]))
for i in range(test_set_n.shape[0]):
    test_targets.append(int(test_set_n[i][3]))

(10581, 4)
(2350, 4)
(288, 4)


In [ ]:
# Training constant
N_WAY = 5
K_SHOT = 10

# Training set
train_maml_sampler = TaskBatchSampler(train_targets,
                                           include_query=True,
                                           N_way=N_WAY,
                                           K_shot=K_SHOT,
                                           batch_size=64)
train_maml_loader = data.DataLoader(train_set,
                                         batch_sampler=train_maml_sampler,
                                         collate_fn=train_maml_sampler.get_collate_fn(),
                                         num_workers=1)

# Validation set
val_maml_sampler = TaskBatchSampler(val_targets,
                                         include_query=True,
                                         N_way=N_WAY,
                                         K_shot=K_SHOT,
                                         batch_size=16,  # We do not update the parameters, hence the batch size is irrelevant here
                                         shuffle=False)
val_maml_loader = data.DataLoader(val_set,
                                       batch_sampler=val_maml_sampler,
                                       collate_fn=val_maml_sampler.get_collate_fn(),
                                       num_workers=1)

In [ ]:
# def split_batch(imgs, targets, yprAngles):
#     N = 2
#     temp_imgs = [iter(imgs)] * N
#     # splt_imgs = list(zip(*temp_imgs))
#     support_imgs, query_imgs = list(zip(*temp_imgs))[0]
#     print(support_imgs)
#     temp_targets = [iter(targets)] * N
#     support_targets, query_targets = list(zip(*temp_targets))[0]

#     temp_gt = [iter(yprAngles)] * N
#     support_gt, query_gt = list(zip(*temp_gt))[0]

#     return support_imgs, query_imgs, support_targets, query_targets, support_gt, query_gt

In [ ]:
def split_batch(imgs, targets, yprAngles):
    support_imgs, query_imgs = imgs.chunk(2, dim=0)
    support_targets, query_targets = targets.chunk(2, dim=0)
    support_gt, query_gt = yprAngles.chunk(2, dim=0)
    return support_imgs, query_imgs, support_targets, query_targets, support_gt, query_gt

In [ ]:
import torch
import torch.nn as nn

class NetSimple(nn.Module):
    def __init__(self, num_feature=200, num_hidden=1000, num_output=3):
        super(NetSimple, self).__init__()

        self.num_feature = num_feature
        self.num_hidden = num_hidden
        self.num_output = num_output

        self.linear1 = nn.Sequential(
            torch.nn.Linear(self.num_feature, self.num_hidden),
            torch.nn.Dropout(p=0.25),
            torch.nn.LeakyReLU(0.1),
            torch.nn.BatchNorm1d(self.num_hidden, eps=1e-05, momentum=0.01, affine=True)
        )

        self.linear2 = nn.Sequential(
            torch.nn.Linear(self.num_hidden, 1000),
            torch.nn.Dropout(p=0.25),
            torch.nn.LeakyReLU(0.1),
            torch.nn.BatchNorm1d(1000, eps=1e-05, momentum=0.01, affine=True)
        )

        self.linear3 = nn.Sequential(
            torch.nn.Linear(1000, 1000),
            torch.nn.Dropout(p=0.25),
            torch.nn.LeakyReLU(0.1),
            torch.nn.BatchNorm1d(1000, eps=1e-05, momentum=0.01, affine=True)
        )

        self.linear4 = nn.Sequential(
            torch.nn.Linear(1000, 1000),
            torch.nn.Dropout(p=0.25),
            torch.nn.LeakyReLU(0.1),
            torch.nn.BatchNorm1d(1000, eps=1e-05, momentum=0.01, affine=True)
        )

        self.output = nn.Sequential(
            torch.nn.Linear(1000, self.num_output),
        )

    def forward(self, x):
        var_size = x.data.shape[0]
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.output(x)
        return x

In [ ]:
from torchsummary import summary
model = NetSimple()

In [ ]:
import higher
import time

In [ ]:
# for i in range(1):
#   a, b, c = next(iter(train_maml_loader))[63]
#   print(a)

In [ ]:
# len(train_set) // 64

In [ ]:
# n_train_iter = len(train_set) // 64

# for batch_idx in range(1):
#     start_time = time.time()
#     batch = next(iter(train_maml_loader))
#     for batch_item in batch:
#         # Sample a batch of support and query images and labels.
#         imgs, targets, yprAngles = batch_item
#         support_imgs, query_imgs, support_targets, query_targets, support_gt, query_gt = split_batch(imgs, targets, yprAngles)


In [ ]:
# import pandas as pd

# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# lr=0.01
# lr_inner=0.01
# train_batch_size = 64
# val_batch_size = 1
# meta_optimizer = optim.SGD(model.parameters(), lr=lr)
# inner_optimizer = torch.optim.SGD(model.parameters(), lr=lr_inner)
# meta_loss = nn.L1Loss()

# def train(meta_opt, inner_optimizer, epoch, log):

#     db = train_maml_loader
#     net = model
#     net.train()
#     n_train_iter = len(train_set) // train_batch_size


#     for batch_idx in range(n_train_iter):
#         start_time = time.time()
#         # Sample a batch of support and query images and labels.
#         batch = next(iter(train_maml_loader))
#         for batch_item in batch:
#             # Sample a batch of support and query images and labels.
#             imgs, targets, yprAngles = batch_item
#             support_imgs, query_imgs, support_targets, query_targets, support_gt, query_gt = split_batch(imgs, targets, yprAngles)

#             task_num, setsz = support_imgs.size()
#             num_inner_steps = 5
#             qry_losses = []
#             qry_accs = []

#             meta_opt.zero_grad()

#             # for i in range(task_num):
#             with higher.innerloop_ctx(
#                 net, inner_optimizer, copy_initial_weights=False
#             ) as (fnet, diffopt):
#                 for _ in range(num_inner_steps):
#                     preds = net(support_imgs)
#                     spt_loss = meta_loss(preds, support_gt)
#                     diffopt.step(spt_loss)

#                 preds_query = net(query_imgs)
#                 qry_loss = meta_loss(preds_query, query_gt)
#                 qry_losses.append(qry_loss.detach())
#                 qry_loss.backward()

#             meta_opt.step()
#             qry_losses = sum(qry_losses) / len(qry_losses)

#             iter_time = time.time() - start_time
#             i = epoch + float(batch_idx) / n_train_iter
#             if (batch_idx+1) % 10 == 0:
#                 print(
#                     f'[Epoch {i:.2f}] Train Loss: {qry_losses:.2f} | Time: {iter_time:.2f}'
#                 )

#             log.append({
#                 'epoch': i,
#                 'loss': qry_losses,
#                 'mode': 'train',
#                 'time': time.time(),
#             })

# def plot(log):
#     df = pd.DataFrame(log)
#     train_df = df[df['mode'] == 'train']
#     plt.plot(train_df['epoch'], train_df['loss'], label='Train')
#     plt.show()

# log = []
# for epoch in range(2):
#   train(meta_optimizer,inner_optimizer, epoch, log)
#   plot(log)



In [ ]:
# def plot(log):
#     df = pd.DataFrame(log)
#     train_df = df[df['mode'] == 'train']
#     plt.plot(train_df['epoch'], train_df['loss'], label='Train')
#     plt.show()

# plot(log)

In [ ]:
class MAMLNetwork(pl.LightningModule):

    def __init__(self, lr, lr_inner, num_inner_steps):
        """
        Inputs
            lr - Learning rate of the outer loop Adam optimizer
            lr_inner - Learning rate of the inner loop SGD optimizer
            num_inner_steps - Number of inner loop updates to perform
        """
        super().__init__()
        self.save_hyperparameters()
        # self.model = Net()
        self.model = NetSimple()
        self.mae_criterion = nn.L1Loss()
        self.training_losses = []
        self.val_losses = []
        self.automatic_optimization = False


    def configure_optimizers(self):
        opt = torch.optim.SGD(self.parameters(), lr=0.01)
        return [opt]

    def run_model(self, imgs, support_gt):
        # Execute a model with given output layer weights and inputs
        preds = self.model(imgs)
        loss = self.mae_criterion(preds, support_gt)
        return loss, preds

    def adapt_few_shot(self, support_imgs, support_targets, support_gt):
        # Create inner-loop model and optimizer
        local_model = deepcopy(self.model)
        local_model.train()
        local_optim = optim.SGD(local_model.parameters(), lr=self.hparams.lr_inner)
        local_optim.zero_grad()


        # Optimize inner loop model on support set
        for _ in range(self.hparams.num_inner_steps):
            # Determine loss on the support set
            loss, pred = self.run_model(local_model, support_imgs, support_gt)
            # Calculate gradients and perform inner loop update
            loss.backward()
            local_optim.step()
            # Reset gradients
            local_optim.zero_grad()

        return local_model

    def outer_loop(self, batch, mode="train"):
        accuracies = []
        losses = []
        self.model.zero_grad()
        qry_loss = torch.tensor(0., device=self.device)
        inner_loss = torch.tensor(0., device='cpu')

        # Determine gradients for batch of tasks
        for task_batch in batch:
            imgs, targets, yprAngles = task_batch
            support_imgs, query_imgs, support_targets, query_targets, support_gt, query_gt = split_batch(imgs, targets, yprAngles)

            # Perform inner loop adaptation

            inner_optimizer = optim.SGD(self.parameters(), lr=self.hparams.lr_inner)

            track_higher_grads = True if mode == 'train' else False

            with higher.innerloop_ctx(
                self.model, inner_optimizer, copy_initial_weights=False, track_higher_grads=track_higher_grads
            ) as (fnet, diffopt):
                for _ in range(self.hparams.num_inner_steps):
                    spt_loss, pred = self.run_model(support_imgs, support_gt)
                    diffopt.step(spt_loss)

                qry_loss, preds = self.run_model(query_imgs, query_gt)

                if mode == 'train':
                    self.manual_backward(qry_loss)

                losses.append(qry_loss.detach())


        # Perform update of base model
        if mode == "train":
            outer_optimizer=self.optimizers()
            outer_optimizer.step()

        self.logger.log_metrics({f"{mode}_loss_custom" : sum(losses) / len(losses)}, step=self.current_epoch)
        self.log(f"{mode}_loss", sum(losses) / len(losses))

    def training_step(self, batch, batch_idx):
        self.outer_loop(batch, mode="train")
        return None

    def validation_step(self, batch, batch_idx):
        # Validation requires to finetune a model, hence we need to enable gradients
        torch.set_grad_enabled(True)
        self.outer_loop(batch, mode="val")
        torch.set_grad_enabled(False)


In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

pl.__version__

Device: cuda:0


'1.7.2'

In [ ]:

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
from numpy.lib.twodim_base import triu_indices_from
def train_model(model_class, train_loader, val_loader, **kwargs):

    # tune hyperparameter
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, model_class.__name__),
                         gpus=1 if str(device) == "cuda:0" else 0, max_epochs = 250, log_every_n_steps = 3, logger=True,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="val_loss"),
                                    LearningRateMonitor("epoch")])

    trainer.logger._default_hp_metric = None

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(
        CHECKPOINT_PATH, model_class.__name__ + ".ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        # Automatically loads the model with the saved hyperparameters
        model = model_class.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)  # To be reproducable
        model = model_class(**kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = model_class.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)  # Load best checkpoint after training

    return model

In [ ]:
protomaml_model = train_model(MAMLNetwork,
                              lr=0.01,
                              lr_inner=0.01,
                              num_inner_steps=8,  # Often values between 1 and 10
                              train_loader=train_maml_loader,
                              val_loader=val_maml_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type      | Params
--------------------------------------------
0 | model         | NetSimple | 3.2 M 
1 | mae_cr

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1898: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=3). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=250` reached.


In [ ]:
%tensorboard --logdir 'JOURNAL_ARTICLE_SECOND_ORDER'

In [ ]:
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

log_dir = "../JOURNAL_ARTICLE_VANILLA/MAMLNetwork/lightning_logs/lightning_logs/version_0/"

event_accumulator = EventAccumulator(log_dir)
# event_accumulator.Reload()

events = event_accumulator.Scalars("train_loss")
x = [x.step for x in events]
y = [x.value for x in events]

df = pd.DataFrame({"step": x, "train_loss": y})
df.to_csv("train_loss.csv")
print(df)

In [ ]:

CHECKPOINT_PATH = '../headpose_checkpoint_five_shot_new/MAMLNetwork/lightning_logs/version_0/checkpoints/'

pretrained_filename = os.path.join(CHECKPOINT_PATH + "epoch=73-step=221.ckpt")

if os.path.isfile(pretrained_filename):
  # Automatically loads the model with the saved hyperparameters
  model = MAMLNetwork.load_from_checkpoint(pretrained_filename)
